### Solutions to the problems of Chapter 8 
## Trees 

##### Implementations to the Tree ADT, Binary Tree, Linked Binary Tree, Array Binary Tree

In [1]:
import math 
import time
import pygame
from collections import deque
from itertools import groupby

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Tree:
    class Position: 
        def element(self): 
            raise NotImplementedError('must be implemented by subclass') 
        def __eq__(self, other):
            raise NotImplementedError('must be implemented by subclass') 
        def __ne__(self, other):
            return not (self == other)
    
    ## accessor methods
    
    def root(self):
        raise NotImplementedError('must be implemented by subclass')
    def parent(self, p):
        raise NotImplementedError('must be implemented by subclass') 
    def is_root(self, p):
        return self.root() == p 
    def num_children(self, p):
        raise NotImplementedError('must be implemented by subclass')
    def children(self, p):
        raise NotImplementedError('must be implemented by subclass')
    def is_leaf(self, p):
        return self.num_children(p) == 0
    def __len__(self):
        raise NotImplementedError('must be implemented by subclass') 
    def is_empty(self):
        return len(self) == 0
    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p)) 
        
    def height(self, p = None):
        def _height(p): 
            if self.is_leaf(p):
                return 0
            else:
                return 1 + max(_height(n) for n in self.children(p))
            
        if p is None:
            p = self.root()
        return _height(p)
    
        
class BinaryTree(Tree): 
    def left(self, p):
        raise NotImplementedError('must be implemented by subclass')
    def right(self, p):
        raise NotImplementedError('must be implemented by subclass') 
        
    def sibling(self, p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
            
    def children(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)
            
    def num_children(self, p):
        num_child = 0 
        if self.left(p) is not None:
            num_child += 1 
        if self.right(p) is not None:
            num_child += 1
        return num_child
    
    def num_descendants(self, p):
        num = 0 
        q = deque() 
        q.append(p)
        while len(q) != 0:
            node = q.popleft()
            num += self.num_children(node)
            for child in self.children(node): 
                    q.append(child)
        return num  

class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = '_element', '_parent', '_left', '_right'
        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent 
            self._left = left
            self._right = right
    
    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container 
            self._node = node
        def element(self):
            return self._node._element 
        def __eq__(self, other): 
            return type(other) is type(self) and self._node is other._node      
        
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node  
    
    def _make_position(self, node): 
        return self.Position(self, node) if node is not None else None 
    
    def __init__(self):
        self._root = None
        self._size = 0
        
    def __len__(self):
        return self._size 
    
    def root(self):
        return self._make_position(self._root)
    
    def parent(self, p):
        return self._make_position(self._validate(p)._parent)
    
    def left(self, p):
        return self._make_position(self._validate(p)._left)
    
    def right(self, p):
        return self._make_position(self._validate(p)._right)
    
    def size(self):
        return self._size 
    
    def _add_root(self, e):
        if self._root is not None: raise ValueError('root already exists !')
        self._root = self._Node(e)
        self._size = 1
        return self._make_position(self._root)
    
    def _add_left(self, p, e):
        positioned_node = self._validate(p)
        if positioned_node._left is not None: raise ValueError('left position is not empty !')
        positioned_node._left = self._Node(e, parent = positioned_node)
        self._size += 1 
        return self._make_position(positioned_node._left)
    
    def _add_right(self, p, e):
        positioned_node = self._validate(p)
        if positioned_node._right is not None: raise ValueError('left position is not empty !')
        positioned_node._right = self._Node(e, parent = positioned_node)
        self._size += 1
        return self._make_position(positioned_node._right) 
    
    def _replace(self, p, e):
        positioned_node = self._validate(p)
        old_value = positioned_node._element
        positioned_node._element = e
        return old_value 
    
    def _delete(self, p):
        node = self._validate(p)
        if self.num_children(p)==2: raise ValueError('p has 2 children')
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent 
        if node is self._root:
            self._root = child 
        else:
            parent = node._parent
            if child == node._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element 
    
    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('p must be a leaf position')
        if type(self) is type(t1) is type(t2): raise TypeError('Invalid types for trees')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node 
            node._left = t1._root 
            t1._root = None  # ??? 
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root 
            t2._root = None
            t2._size = 0
            
    def _delete_subtree(self, p):
        parent = self.parent(p)
        parent_node = self._validate(parent) 
        num_desc = self.num_descendants(p)
        
        if self.left(parent) == p:
            parent_node._left = None 
        else:
            parent_node._right = None
        self._size -= num_desc +1
        
        deleted_node = self._validate(p)
        deleted_node._parent = None
        
    def _swap(self, p, q): 
        first_node = self._validate(p)
        second_node = self._validate(q)
        
        if first_node._parent is second_node._parent:
            first_node._parent._left, first_node._parent._right = first_node._parent._right, first_node._parent._left
        
        else:
            #swap parents
            first_node._parent, second_node._parent = second_node._parent, first_node._parent
            # check if first was left, put second in left else do put in right 
            if second_node._parent._left == first_node:
                second_node._parent._left = second_node
            else:
                second_node._parent._right = second_node 

            if first_node._parent._left == second_node: 
                first_node._parent._left = first_node
            else:
                first_node._parent._right = first_node 
            
    def __iter__(self):
        for position in self.positions():
            yield position.element()
            
    def preorder(self):
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()): 
                yield p 
    def _subtree_preorder(self, p): 
        yield p
        for c in self.children(p): 
            for other in self._subtree_preorder(c):
                yield other
                
    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p
    def _subtree_postorder(self, p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p 
        
    def BFS(self):
        if not self.is_empty():
            frontier = deque()
            frontier.append(self.root())
            while len(frontier) != 0:
                p = frontier.popleft()
                yield p 
                for c in self.children(p): 
                    frontier.append(c)
                    
    def inorder(self): 
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p 
                
    def _subtree_inorder(self, p):
        if self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)): 
                yield other
        yield p
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other
                
    def positions(self, pre=False, ino = False, post=False, BFS=False):
        if pre and not ino and not post and not BFS:
            return self.preorder()
        elif ino and not post and not pre and not BFS:
            return self.inorder()
        elif post and not pre and not ino and not BFS:
            return self.postorder()
        elif BFS and not pre and not ino and not post:
            return self.BFS() 
        else:
            raise ValueError('Only one argument required')
            
            
    def __str__(self):
        print_list = [] 
        def print_tree(position, print_list, depth):
            print_list.append('-'*depth+'>'*bool(depth)+ '('+str(position.element())+')') 
            for c in self.children(position):
                print_tree(c, print_list, depth + 3)
            return ''.join(item+'\n' for item in print_list)
        
        if not self.is_empty():
            result = print_tree(self.root(), print_list, depth = 0)
            return result
        else:
            return "" 

In [3]:
bt = LinkedBinaryTree()
bt._add_root(1)
root = bt.root()
bt._add_left(bt.root(), 2)
bt._add_right(bt.root(), 3)
bt._add_left(bt.right(bt.root()), 4)
print(bt.is_empty())
print([item.element() for item in list(bt.positions(BFS = True))])

print(bt.num_descendants(root))


False
[1, 2, 3, 4]
3


****Reinforcement Problems*** 

R-8.5 Describe an algorithm, relying only on the BinaryTree operations, that counts the number of leaves in a binary tree that are the left child of their respective parent.

In [4]:
# naive solution:
# traverse using your favorite traversal, when you visit a node, if it's leaf and it's left to it's parent increment the node 

# counter = 0
# for node in T.positions():
#     if T.is_leaf(node) and T.parent(node).left == node:
#        counter +=1 

R-8.6 Let T be an n-node binary tree that may be improper. Describe how to represent T by means of a proper binary tree T' with O(n) nodes.

In [5]:
# for node in T.positions():
#     if not T.is_leaf(node) and len(T.children(node)) !=2:
#             if T.left(node) is not None:
#                 T.add_left(element)
#             else:
#                 T.add_right(element)

R-8.10 Give a direct implementation of the num children method within the class BinaryTree.

In [6]:
# added to first code segment in the notebook 

R-8.15 The LinkedBinaryTree class provides only nonpublic versions of the update methods discussed on page 319. Implement a simple subclass named MutableLinkedBinaryTree that provides public wrapper functions for each of the inherited nonpublic update methods.

In [7]:
class MutableLinkedBinaryTree(LinkedBinaryTree): 
    def add_root(self, e): 
        return self._add_root(e)
    
    def add_left(self, p, e):
        return self._add_left(p, e)
        
    def add_right(self, p, e): 
        return self._add_right(p, e)
        
    def replace(self, p, e):
        return self._replace(p, e)
    
    def delete(self, p):
        return self._delete(p)
    
    def attach(self, p, t1, t2):
        self._attach(p, t1, t2)

R-8.18 Let T be a binary tree with n positions that is realized with an array representation A, and let f() be the level numbering function of the positions of T , as given in Section 8.3.2. Give pseudo-code descriptions of each of the methods root, parent, left, right, is leaf, and is root.

In [8]:
# suppose arr =[] cotains the elements of the tree 
# root():
#    return arr[0]
# parent(i):
#    return arr[floor((i-1)/2)]
# left(i):
#   if i is even: 
#      return arr[i-1]
# right(i):
#   if i is odd: 
#      return arr[i+1]
#  is_leaf(i):
#    if arr[2*(i+1)+1] is None and arr[2*(i+1)+2] is None return True else False 
#  is_root(i):
#    if arr[0] is None return True else False


R-8.19 Our definition of the level numbering function f(p), as given in Section 8.3.2, began with the root having number 0. Some authors prefer to use a level numbering g(p) in which the root is assigned number 1, because it simplifies the arithmetic for finding neighboring positions. Redo Exercise R-8.18, but assuming that we use a level numbering g(p) in which the root is assigned number 1

In [9]:
# trivial: same as before just shift all the math by -1 

R-8.26 The collections.deque class supports an extend method that adds a collection of elements to the end of the queue at once. Reimplement the breadthfirst method of the Tree class to take advantage of this feature.

In [10]:
class ExtendQueueBFSLinkedBinaryTree(LinkedBinaryTree):
    def BFS(self):
        if not self.is_empty():
            frontier = deque()
            frontier.append(self.root())
            while len(frontier) != 0:
                p = frontier.popleft()
                yield p
                frontier.extend(self.children(p)) 
                
bt = ExtendQueueBFSLinkedBinaryTree()
bt._add_root(1)
root = bt.root()
bt._add_left(bt.root(), 2)
bt._add_right(bt.root(), 3)
bt._add_left(bt.right(bt.root()), 4)
print([item.element() for item in list(bt.positions(BFS = True))])


[1, 2, 3, 4]


R-8.29 Describe, in pseudo-code, an algorithm for computing the number of descendants of each node of a binary tree. The algorithm should be based on the Euler tour traversal.

In [11]:
# Algorithm eulertour(T, p, counter = 0):
#     perform the “pre visit” action for position p
#     for each child c in T.children(p) do
#         eulertour(T, c, counter+=1)  {recursively tour the subtree rooted at c}
        
#     perform the “post visit” action for position p
#       return counter

R-8.30 The build expression tree method of the ExpressionTree class requires input that is an iterable of string tokens. We used a convenient example, (((3+1)x4)/((9-5)+2)) , in which each character is its own token, so that the string itself sufficed as input to build expression tree. In general, a string, such as (35 + 14) , must be explicitly tokenized into list [ ( , 35 , + , 14 , ) ] so as to ignore whitespace and to recognize multidigit numbers as a single token. Write a utility method, tokenize(raw), that returns such a list of tokens for a raw string.

In [12]:
expression = '(35 + 14)'
tokens = [''.join(j) for k, j in groupby(expression, str.isdigit)]
tokens

['(', '35', ' + ', '14', ')']

****Creativity Problems***

C-8.35 Two ordered trees T' and T'' are said to be isomorphic if one of the following holds:
- Both T' and T'' are empty.
-  The roots of T' and T'' have the same number k ≥ 0 of subtrees, and
the ith such subtree of T' is isomorphic to the ith such subtree of T'' 
for i = 1,...,k.


Design an algorithm that tests whether two given ordered trees are isomorphic. What is the running time of your algorithm?

In [13]:
def isIsomorphic(n1, n2): 
    if n1 is None and n2 is None: 
        return True
    if n1 is None or n2 is None: 
        return False
    if n1.data != n2.data : 
        return False
    return ((isIsomorphic(n1.left, n2.left)and 
            isIsomorphic(n1.right, n2.right))or
            (isIsomorphic(n1.left, n2.right)and 
            isIsomorphic(n1.right, n2.left)))

C-8.38 Add support in LinkedBinaryTree for a method, delete subtree(p), that removes the entire subtree rooted at position p, making sure to maintain the count on the size of the tree. What is the running time of your implementation?

In [14]:
bt = LinkedBinaryTree()
bt._add_root(1)
root = bt.root()
bt._add_left(bt.root(), 2)
bt._add_right(bt.root(), 3)
bt._add_left(bt.right(bt.root()), 4)
bt._add_right(bt.right(bt.root()), 5)

print(bt)
print(f'size of the tree  initially ={bt._size}')
print(bt.num_descendants(root))
print(bt.num_children(root))

bt._delete_subtree(bt.right(root))
print(bt)
print(bt.num_descendants(root))
print(bt.num_children(root))
print(bt._size)

(1)
--->(2)
--->(3)
------>(4)
------>(5)

size of the tree  initially =5
4
2
(1)
--->(2)

1
1
2


C-8.39 Add support in LinkedBinaryTree for a method, swap(p,q), that has the effect of restructuring the tree so that the node referenced by p takes the place of the node referenced by q, and vice versa. Make sure to properly handle the case when the nodes are adjacent.

In [15]:
bt = LinkedBinaryTree()
bt._add_root(1)
root = bt.root()
bt._add_left(bt.root(), 2)
bt._add_right(bt.root(), 3)
bt._add_left(bt.right(bt.root()), 7)
bt._add_right(bt.right(bt.root()), 8)
bt._add_left(bt.left(bt.root()), 4)
bt._add_left(bt.left(bt.left(bt.root())), 5)           
bt._add_right(bt.left(bt.left(bt.root())), 6)
print("Original Tree:")
print(bt)

print("Re-strcutred Tree:")
bt._swap(bt.left(bt.root()), bt.right(bt.right(bt.root())))
print(bt)

Original Tree:
(1)
--->(2)
------>(4)
--------->(5)
--------->(6)
--->(3)
------>(7)
------>(8)

Re-strcutred Tree:
(1)
--->(8)
--->(3)
------>(7)
------>(2)
--------->(4)
------------>(5)
------------>(6)



In [16]:
bt = LinkedBinaryTree()
bt._add_root(1)
root = bt.root()
bt._add_left(bt.root(), 2)
bt._add_right(bt.root(), 3)
bt._add_left(bt.right(bt.root()), 7)
bt._add_right(bt.right(bt.root()), 8)
bt._add_left(bt.left(bt.root()), 4)
bt._add_left(bt.left(bt.left(bt.root())), 5)           
bt._add_right(bt.left(bt.left(bt.root())), 6)
print("Original Tree:")
print(bt)

print("Re-strcutred Tree:")
bt._swap(bt.left(bt.root()), bt.right(bt.root()))
print(bt)

Original Tree:
(1)
--->(2)
------>(4)
--------->(5)
--------->(6)
--->(3)
------>(7)
------>(8)

Re-strcutred Tree:
(1)
--->(3)
------>(7)
------>(8)
--->(2)
------>(4)
--------->(5)
--------->(6)



c-8.41 and C-8.42 are trivial

C-8.44 Give an efficient algorithm that computes and prints, for every position p of a tree T , the element of p followed by the height of p’s subtree.

In [17]:
bt = LinkedBinaryTree()
bt._add_root('a')
root = bt.root()
bt._add_left(bt.root(), 'b')
bt._add_right(bt.root(), 'c')
bt._add_left(bt.right(bt.root()), 'd')
bt._add_right(bt.right(bt.root()), 'e')
bt._add_right(bt.right(bt.right(bt.root())), 'f')
print("Original Tree:")
print(bt)

def get_hight(tree): 
    root = tree.root()
    nodes_hights = []
    def traverse(position):
        if tree.is_leaf(position):
            nodes_hights.append((position.element(), 0))
            return 0 
        value  = 1 + max(traverse(node) for node in tree.children(position)) 
        nodes_hights.append((position.element(), value))
        return value 
    traverse(root)
    return nodes_hights

Original Tree:
(a)
--->(b)
--->(c)
------>(d)
------>(e)
--------->(f)



In [18]:
get_hight(bt)

[('b', 0), ('d', 0), ('f', 0), ('e', 1), ('c', 2), ('a', 3)]

C-8.45 Give an O(n)-time algorithm for computing the depths of all positions of a tree T , where n is the number of nodes of T .

In [19]:
def get_depth(tree): 
    root = tree.root()
    nodes_depths = [] 
    def traverse(position, depth):
        if position:
            nodes_depths.append((position.element(), depth))
            for child in tree.children(position):
                traverse(child, depth+1)
    traverse(root, depth = 0)
    return nodes_depths

In [20]:
z = get_depth(bt)
print(z)
zz = sum(element[1] for element in z)

print(zz)

[('a', 0), ('b', 1), ('c', 1), ('d', 2), ('e', 2), ('f', 3)]
9


C-8.46 The path length of a tree T is the sum of the depths of all positions in T. Describe a linear-time method for computing the path length of a tree T .

In [21]:
def get_path_lenght(tree): 
    root = tree.root()
    paths_length = [0]
    def traverse(position, depth):
        if position:
            paths_length[0]+=depth
            for child in tree.children(position):
                traverse(child, depth+1) 
    traverse(root, depth = 0)
    return paths_length[0]


In [22]:
get_path_lenght(bt)

9

C-8.47 The balance factor of an internal position p of a proper binary tree is the difference between the heights of the right and left subtrees of p. Show how to specialize the Euler tour traversal of Section 8.4.6 to print the balance factors of all the internal nodes of a proper binary tree.

In [23]:
def get_balance_factor(tree):
    root = tree.root()
    b_factors = []
    def euler_tour(position):
        if position: 
            if tree.is_leaf(position):
                return 0 
            else:
                # pre_visit : do nothing 
                a = euler_tour(tree.left(position))
                b = euler_tour(tree.right(position))
                # post_vist : get diff 
                h = 1 + max(a, b)
                diff = abs(a-b)
                b_factors.append((position.element(), diff))
                return h
        else:
            return 0 
    euler_tour(root)
    return b_factors

In [24]:
get_balance_factor(bt)

[('e', 0), ('c', 1), ('a', 2)]

C-8.48 Given a proper binary tree T , define the reflection of T to be the binary tree T' such that each node v in T is also in T', but the left child of v in T is v’s right child in T' and the right child of v in T is v’s left child in T'. Show that a preorder traversal of a proper binary tree T is the same as the
postorder traversal of T ’s reflection, but in reverse order.

In [25]:
print(bt)

(a)
--->(b)
--->(c)
------>(d)
------>(e)
--------->(f)



In [26]:
def reflect_tree(tree, position):
    if position is not None:
        node = tree._validate(position)
        node._left, node._right = node._right, node._left 
        for child in tree.children(position): 
            reflect_tree(tree, child) 

In [27]:
print([item.element() for item in bt.positions(pre = True)])            
reflect_tree(bt, bt.root())

['a', 'b', 'c', 'd', 'e', 'f']


In [28]:
print([item.element() for item in bt.positions(post = True)])

['f', 'e', 'd', 'c', 'b', 'a']


C-8.50 Design algorithms for the following operations for a binary tree T :
    - preorder next(p): Return the position visited after p in a preorder traversal of T (or None if p is the last node visited).
    - inorder next(p): Return the position visited after p in an inorder traversal of T (or None if p is the last node visited).
    - postorder next(p): Return the position visited after p in a postorder traversal of T (or None if p is the last node visited).
What are the worst-case running times of your algorithms?

In [29]:
## Worst case running time for all the algorithms is O(n)
## space complexity is o(1)

In [30]:
bt = LinkedBinaryTree()
bt._add_root('a')
root = bt.root()
bt._add_left(bt.root(), 'b')
bt._add_right(bt.root(), 'c')
bt._add_left(bt.left(bt.root()), 'd')
bt._add_right(bt.left(bt.root()), 'e')
bt._add_left(bt.right(bt.root()), 'f')
bt._add_right(bt.right(bt.root()), 'g')

bt._add_left(bt.left(bt.left(bt.root())), 'h')
bt._add_right(bt.left(bt.left(bt.root())), 'i')

bt._add_left(bt.right(bt.left(bt.root())), 'j')
bt._add_right(bt.right(bt.left(bt.root())), 'k')

bt._add_left(bt.left(bt.right(bt.root())), 'l')
bt._add_right(bt.left(bt.right(bt.root())), 'm')

bt._add_left(bt.right(bt.right(bt.root())), 'n')
bt._add_right(bt.right(bt.right(bt.root())), 'o')


bt._add_right(bt.left(bt.right(bt.right(bt.root()))), 'XX')
print("Original Tree:")
print(bt)


Original Tree:
(a)
--->(b)
------>(d)
--------->(h)
--------->(i)
------>(e)
--------->(j)
--------->(k)
--->(c)
------>(f)
--------->(l)
--------->(m)
------>(g)
--------->(n)
------------>(XX)
--------->(o)



In [31]:
print([item.element() for item in bt.positions(pre= True)])
print([item.element() for item in bt.positions(post= True)])
print([item.element() for item in bt.positions(ino= True)])

['a', 'b', 'd', 'h', 'i', 'e', 'j', 'k', 'c', 'f', 'l', 'm', 'g', 'n', 'XX', 'o']
['h', 'i', 'd', 'j', 'k', 'e', 'b', 'l', 'm', 'f', 'XX', 'n', 'o', 'g', 'c', 'a']
['h', 'd', 'i', 'b', 'j', 'e', 'k', 'a', 'l', 'f', 'm', 'c', 'n', 'XX', 'g', 'o']


In [32]:
def get_next(tree, position, traversal):
    if traversal == 'pre': 
        if not tree.is_leaf(position):
            if tree.left(position) is not None:
                return tree.left(position)
            else:
                return tree.right(position)
        parent = tree.parent(position)
        while(parent):
            if parent is not None: 
                if tree.right(parent) != position:
                    return tree.right(parent)
                    break 
                position = parent
            parent = tree.parent(position)
            
    if traversal == 'post':
        if tree.is_root(position):
            return
        if tree.right(tree.parent(position)) == position: 
            return tree.parent(position)
        if tree.sibling(position) is not None:
            sibling = tree.sibling(position)
            if tree.is_leaf(sibling):
                return sibling
            else:
                while(True): 
                    if tree.left(sibling) is not None:
                        current = tree.left(sibling)
                    else:
                        current = tree.right(sibling)
                    if tree.is_leaf(current):
                        return current
                    else:
                        sibling = current
        else:
            return tree.parent(position)
        
    if traversal == 'in':
        if tree.is_leaf(position):
            if tree.left(tree.parent(position)) == position:
                return tree.parent(position)
            else: 
                current = position
                parent = tree.parent(position)
                while(True):
                    if tree.right(parent) == current:
                        current = parent
                        if  tree.parent(parent) is not None: 
                            parent = tree.parent(parent)
                        else:
                            return
                    else: 
                        break
                return parent
        if tree.right(position):
            if tree.is_leaf(tree.right(position)):
                return tree.right(position) 
            else:
                current = tree.right(position)
                while(True):
                    if tree.left(current) is not None:
                        current = tree.left(current)
                    else:
                        return current 

In [33]:
item = bt.root()
flag = True
print("next pre-order nodes: ")
while(flag):
    item = get_next(bt, item, traversal='pre')
    if item is not None:
        print(item.element(), end = ', ')
        continue
    flag = False

item = bt.left(bt.left(bt.left(bt.root())))
flag = True
print("")
print("")
print("next post-order nodes: ")
while(flag):
    item = get_next(bt, item, traversal='post')
    if item is not None:
        print(item.element(), end = ', ')
        continue
    flag = False

item = bt.left(bt.left(bt.left(bt.root())))
flag = True
print("")
print("")
print("next In-order nodes: ")
while(flag):
    item = get_next(bt, item, traversal='in')
    if item is not None:
        print(item.element(), end = ', ')
        continue
    flag = False

next pre-order nodes: 
b, d, h, i, e, j, k, c, f, l, m, g, n, XX, o, 

next post-order nodes: 
i, d, j, k, e, b, l, m, f, XX, n, o, g, c, a, 

next In-order nodes: 
d, i, b, j, e, k, a, l, f, m, c, n, XX, g, o, 

C-8.51 To implement the preorder method of the LinkedBinaryTree class, we relied on the convenience of Python’s generator syntax and the yield statement. Give an alternative implementation of preorder that returns an explicit instance of a nested iterator class. (See Section 2.3.4 for discussion
of iterators.

In [34]:
class iterableLinkedBinaryTree(LinkedBinaryTree):         
    def __iter__(self):
        self.root_flag = True 
        self.nxt = self.root()
        return self 
    
    def get_next(self, tree, position):
        if not tree.is_leaf(position):
            if tree.left(position) is not None:
                return tree.left(position)
            else:
                return tree.right(position)
        parent = tree.parent(position)
        while(parent):
            if parent is not None: 
                if tree.right(parent) != position:
                    return tree.right(parent)
                    break 
                position = parent
            parent = tree.parent(position)

    def __next__(self):
        if self.root_flag: 
            self.root_flag = False
            return self.root()
        if not self.root_flag: 
            self.nxt = self.get_next(self, self.nxt)
            if self.nxt is None:
                raise StopIteration
            return self.nxt

In [35]:
bt = iterableLinkedBinaryTree() 
bt._add_root('a')
root = bt.root()
bt._add_left(bt.root(), 'b')
bt._add_right(bt.root(), 'c')
bt._add_left(bt.left(bt.root()), 'd')
bt._add_right(bt.left(bt.root()), 'e')
bt._add_left(bt.right(bt.root()), 'f')
bt._add_right(bt.right(bt.root()), 'g')

bt._add_left(bt.left(bt.left(bt.root())), 'h')
bt._add_right(bt.left(bt.left(bt.root())), 'i')

bt._add_left(bt.right(bt.left(bt.root())), 'j')
bt._add_right(bt.right(bt.left(bt.root())), 'k')

bt._add_left(bt.left(bt.right(bt.root())), 'l')
bt._add_right(bt.left(bt.right(bt.root())), 'm')

bt._add_left(bt.right(bt.right(bt.root())), 'n')
bt._add_right(bt.right(bt.right(bt.root())), 'o')


bt._add_right(bt.left(bt.right(bt.right(bt.root()))), 'XX')
print("Original Tree:")
print(bt)


Original Tree:
(a)
--->(b)
------>(d)
--------->(h)
--------->(i)
------>(e)
--------->(j)
--------->(k)
--->(c)
------>(f)
--------->(l)
--------->(m)
------>(g)
--------->(n)
------------>(XX)
--------->(o)



In [36]:
print("Orignal implementation: ")
print([item.element() for item in bt.positions(pre= True)]) 
print("nested iterator implementation: ")
for item in bt:
    print(item.element(), end = ', ')

Orignal implementation: 
['a', 'b', 'd', 'h', 'i', 'e', 'j', 'k', 'c', 'f', 'l', 'm', 'g', 'n', 'XX', 'o']
nested iterator implementation: 
a, b, d, h, i, e, j, k, c, f, l, m, g, n, XX, o, 

C-8.52 Algorithm preorder_draw draws a binary tree T by assigning x- and ycoordinates to each position p such that x(p) is the number of nodes preceding p in the preorder traversal of T and y(p) is the depth of p in T. 
- a. Show that the drawing of T produced by preorder_draw has no pairs
- of crossing edges.
- b. Redraw the binary tree of Figure 8.22 using preorder_draw.

In [404]:
class DrawableBinaryTree(LinkedBinaryTree): 
    def inorder_draw(self):
        counter = [0]
        drawable_items = [] 
        def draw_subtree_inorder(pos, x, y): 
            if self.left(pos) is not None:
                left_child =  self.left(pos)
                draw_subtree_inorder(left_child, x+counter[0], y+15)
            
            counter[0]+=10
            if self.parent(pos) is not None: 
                drawable_items.append([pos, pos.element(), x, y])
            else:
                drawable_items.append([pos, pos.element(), x, y, None])
                                              
            if self.right(pos) is not None:
                right_child = self.right(pos)
                draw_subtree_inorder(right_child, x+counter[0], y+15)
        
        draw_subtree_inorder(self.root(), x = 10, y = 10)
        
        for i, node in enumerate(drawable_items):
            node[2]+=(i*25) +(500-5*node[3])
        for node in drawable_items:
            for target_node in drawable_items:
                if not self.is_root(node[0]) and self.parent(node[0]) == target_node[0]:
                    node.extend([target_node[2], target_node[3]])
        return [(item[1:]) for item in drawable_items]

In [405]:
bt = DrawableBinaryTree() 
bt._add_root('a')
root = bt.root()
bt._add_left(bt.root(), 'b')
bt._add_right(bt.root(), 'c')
bt._add_left(bt.left(bt.root()), 'd')
bt._add_right(bt.left(bt.root()), 'e')
bt._add_left(bt.right(bt.root()), 'f')
bt._add_right(bt.right(bt.root()), 'g')

bt._add_left(bt.left(bt.left(bt.root())), 'h')
bt._add_right(bt.left(bt.left(bt.root())), 'i')

bt._add_left(bt.right(bt.left(bt.root())), 'j')
bt._add_right(bt.right(bt.left(bt.root())), 'k')

bt._add_left(bt.left(bt.right(bt.root())), 'l')
bt._add_right(bt.left(bt.right(bt.root())), 'm')

bt._add_left(bt.right(bt.right(bt.root())), 'n')
bt._add_right(bt.right(bt.right(bt.root())), 'o')


bt._add_right(bt.left(bt.right(bt.right(bt.root()))), 'XX')
# print("Original Tree:")
# print(bt)


In [ ]:
pygame.init() 

# game constants 
display_width = 1440 
display_height = 810 
## colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN  = (0, 255, 0)
BLUE = (0, 0, 255)


gameDisplay = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('Binary Tree')
clock = pygame.time.Clock() 


### text_objects 

def text_objects(text, font, color):
    text_surface = font.render(text, True, color)
    return text_surface, text_surface.get_rect() 

def node_display(text, location):
    large_font = pygame.font.Font('freesansbold.ttf', 50)
    text_surf, text_rect = text_objects(" "+text+" ", large_font, BLACK)
    pygame.draw.rect(text_surf, BLUE, text_rect, 5)
    gameDisplay.blit(text_surf, location)

def arrow(screen, lcolor, tricolor, start, end, trirad = 10, thickness=5):
    rad = math.pi/180 
    pygame.draw.line(screen, lcolor, start, end, thickness)
    rotation = (math.atan2(start[1] - end[1], end[0] - start[0])) + math.pi/2
    pygame.draw.polygon(screen, tricolor, ((end[0] + trirad * math.sin(rotation),
                                        end[1] + trirad * math.cos(rotation)),
                                       (end[0] + trirad * math.sin(rotation - 120*rad),
                                        end[1] + trirad * math.cos(rotation - 120*rad)),
                                       (end[0] + trirad * math.sin(rotation + 120*rad),
                                        end[1] + trirad * math.cos(rotation + 120*rad))))

def main_loop():
    Exit = False
    while not Exit:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                Exit = True

        gameDisplay.fill(WHITE)
        #node_display("Hi", location = (100, 100))
        for node in bt.inorder_draw():
            node_display(node[0], location = (node[1], node[2]*10))
            if len(node) ==5: 
                arrow(gameDisplay, RED, BLACK, (node[3]+25, node[4]*10 + 50), (node[1]+25, node[2]*10))
        pygame.display.update()
        clock.tick(25)

main_loop()
pygame.quit()

<ipython-input-407-63ad8ceca4b2>:35: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pygame.draw.polygon(screen, tricolor, ((end[0] + trirad * math.sin(rotation),
